In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

C:\Users\Sahil Karamkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


#### Pre processing img data 

In [ ]:
train_dir = "cifar-10-img/train"
test_dir = "cifar-10-img/test"
#instances | used to preprocess  data for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,   # normalizes the pixel values of images 
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

# here batch_size is the number of images in each batch

train_batch_size = 5000
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=train_batch_size,
    class_mode='categorical'
)
test_batch_size = 1000
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=test_batch_size,
    class_mode='categorical'
)

Found 40079 images belonging to 10 classes.
Found 9921 images belonging to 10 classes.


#### Selecting only first batch with 5000 images as train and test data

In [3]:
x_train, y_train =  train_generator[0]
x_test, y_test = test_generator[0]

print(len(x_train))
print(len(x_test))

5000
1000


#### a. Load in a pre-trained CNN model trained on a large dataset

In [4]:
# Load VGG16 without top layers
weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(32, 32, 3))

#### b. Freeze parameters (weights) in model’s lower convolutional layers

In [5]:
for layer in base_model.layers:
   layer.trainable = False

#### c. Add custom classifier with several layers of trainable parameters to model

In [ ]:
x = Flatten()(base_model.output) #converts the multi-dimensional  output to 1D
x = Dense(256, activation='relu')(x)  #RELU to introduce non-linearity mitigate the vanishing gradient problem 
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)  

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

#### d. Train classifier layers on training data available for task

In [7]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 208ms/step - accuracy: 0.2084 - loss: 2.1658 - val_accuracy: 0.4360 - val_loss: 1.5878
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 230ms/step - accuracy: 0.4068 - loss: 1.6620 - val_accuracy: 0.5000 - val_loss: 1.4321
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 241ms/step - accuracy: 0.4851 - loss: 1.4704 - val_accuracy: 0.5200 - val_loss: 1.3678
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 16s 204ms/step - accuracy: 0.4954 - loss: 1.4156 - val_accuracy: 0.5090 - val_loss: 1.3584
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 16s 207ms/step - accuracy: 0.5541 - loss: 1.2826 - val_accuracy: 0.5430 - val_loss: 1.3192
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 16s 199ms/step - accuracy: 0.5662 - loss: 1.2295 - val_accuracy: 0.5440 - val_loss: 1.3006
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 16s 199ms/step - accuracy: 0.5936 - loss: 1.1786 - val_accuracy: 0.5390 - val_loss: 1.3083
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 16s 201ms/step - accuracy: 0.5947 - loss: 1.1343 - val_accu

#### e. Fine-tune hyper parameters and unfreeze more layers as needed

In [ ]:
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(32, 32, 3))
# freeze all layers first
for layer in base_model.layers:
   layer.trainable = False
# unfreeze last 4 layers of base model
for layer in base_model.layers[len(base_model.layers) - 4:]:
   layer.trainable = True
# fine-tuning hyper parameters
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# training fine tuned model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
 2/79 ━━━━━━━━━━━━━━━━━━━━ 1:20 1s/step - accuracy: 0.0820 - loss: 2.4009

In [ ]:
import matplotlib.pyplot as plt
predicted_value = model.predict(x_test)

In [ ]:
labels = list(test_generator.class_indices.keys())

In [ ]:
n = 890   #index of test image 
plt.imshow(x_test[n])
print("Preditcted: ",labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(y_test[n])])